In [70]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import pickle
from sklearn.neighbors import KNeighborsClassifier  # Importar KNeighborsClassifier

In [60]:
df = pd.read_csv("../data/sidewalk-cafe-permits.csv")
df.head()

,PERMIT NUMBER,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ISSUED DATE,EXPIRATION DATE,PAYMENT DATE,ADDRESS,ADDRESS NUMBER START,...,WARD PRECINCT,POLICE DISTRICT,LATITUDE,LONGITUDE,LOCATION,Community Areas,Zip Codes,Boundaries - ZIP Codes,Census Tracts,Wards
0,1142666,458934,1,"TINTO & TAPAS, LLC",TINTO & TAPAS,2019-11-25T00:00:00.000,2020-02-29T00:00:00.000,2019-10-29T00:00:00.000,7958 W BELMONT AVE,7958,...,38-30,16.0,41.937859,-87.826255,"{'latitude': '41.937859427847215', 'human_addr...",17.0,22254.0,52.0,457.0,19.0
1,1142290,294900,4,"PIE CAFE, LLC",Pie cafe,2019-10-15T00:00:00.000,2020-02-29T00:00:00.000,2019-10-15T00:00:00.000,5357 N ASHLAND AVE,5357,...,40-26,20.0,41.979717,-87.669565,"{'latitude': '41.979717096739535', 'human_addr...",76.0,22616.0,15.0,564.0,24.0
2,1142132,459163,1,KALIFLOWER 333 NORTH MICHIGAN AVE LLC,KALIFLOWER,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,2019-09-20T00:00:00.000,333 N MICHIGAN AVE,333,...,42-40,1.0,41.887543,-87.624407,"{'latitude': '41.88754294139151', 'human_addre...",38.0,14309.0,42.0,580.0,36.0
3,1142101,384478,1,1732 N MILWAUKEE RESTAURANT LLC,SMALL CHEVAL,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,2019-09-20T00:00:00.000,1732 N MILWAUKEE AVE,1732,...,32-6,14.0,41.912730,-87.681327,"{'latitude': '41.91273041812944', 'human_addre...",25.0,22535.0,1.0,298.0,40.0
4,1142100,456144,1,"THE WHALE CHICAGO, LLC",THE WHALE CHICAGO,2019-09-19T00:00:00.000,2020-02-29T00:00:00.000,2019-09-19T00:00:00.000,2427-2431 N MILWAUKEE AVE,2427,...,1-19,14.0,41.925388,-87.701170,"{'latitude': '41.92538801822599', 'human_addre...",23.0,22535.0,1.0,548.0,41.0


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17998 entries, 0 to 17997
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PERMIT NUMBER           17998 non-null  int64  
 1   ACCOUNT NUMBER          17998 non-null  int64  
 2   SITE NUMBER             17998 non-null  int64  
 3   LEGAL NAME              17998 non-null  object 
 4   DOING BUSINESS AS NAME  17998 non-null  object 
 5   ISSUED DATE             17998 non-null  object 
 6   EXPIRATION DATE         17998 non-null  object 
 7   PAYMENT DATE            17998 non-null  object 
 8   ADDRESS                 17998 non-null  object 
 9   ADDRESS NUMBER START    17998 non-null  object 
 10  ADDRESS NUMBER          17998 non-null  object 
 11  STREET DIRECTION        17998 non-null  object 
 12  STREET                  17998 non-null  object 
 13  STREET TYPE             16920 non-null  object 
 14  CITY                    17998 non-null

In [62]:
df = df.drop(["LEGAL NAME", "DOING BUSINESS AS NAME", "LOCATION", "ADDRESS", "STREET DIRECTION", "WARD"], axis = 1)
df.head()

,PERMIT NUMBER,ACCOUNT NUMBER,SITE NUMBER,ISSUED DATE,EXPIRATION DATE,PAYMENT DATE,ADDRESS NUMBER START,ADDRESS NUMBER,STREET,STREET TYPE,...,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LATITUDE,LONGITUDE,Community Areas,Zip Codes,Boundaries - ZIP Codes,Census Tracts,Wards
0,1142666,458934,1,2019-11-25T00:00:00.000,2020-02-29T00:00:00.000,2019-10-29T00:00:00.000,7958,7958,BELMONT,AVE,...,30.0,38-30,16.0,41.937859,-87.826255,17.0,22254.0,52.0,457.0,19.0
1,1142290,294900,4,2019-10-15T00:00:00.000,2020-02-29T00:00:00.000,2019-10-15T00:00:00.000,5357,5357,ASHLAND,AVE,...,26.0,40-26,20.0,41.979717,-87.669565,76.0,22616.0,15.0,564.0,24.0
2,1142132,459163,1,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,2019-09-20T00:00:00.000,333,333,MICHIGAN,AVE,...,40.0,42-40,1.0,41.887543,-87.624407,38.0,14309.0,42.0,580.0,36.0
3,1142101,384478,1,2019-09-20T00:00:00.000,2020-02-29T00:00:00.000,2019-09-20T00:00:00.000,1732,1732,MILWAUKEE,AVE,...,6.0,32-6,14.0,41.912730,-87.681327,25.0,22535.0,1.0,298.0,40.0
4,1142100,456144,1,2019-09-19T00:00:00.000,2020-02-29T00:00:00.000,2019-09-19T00:00:00.000,2427,2427-2431,MILWAUKEE,AVE,...,19.0,1-19,14.0,41.925388,-87.701170,23.0,22535.0,1.0,548.0,41.0


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17998 entries, 0 to 17997
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PERMIT NUMBER           17998 non-null  int64  
 1   ACCOUNT NUMBER          17998 non-null  int64  
 2   SITE NUMBER             17998 non-null  int64  
 3   ISSUED DATE             17998 non-null  object 
 4   EXPIRATION DATE         17998 non-null  object 
 5   PAYMENT DATE            17998 non-null  object 
 6   ADDRESS NUMBER START    17998 non-null  object 
 7   ADDRESS NUMBER          17998 non-null  object 
 8   STREET                  17998 non-null  object 
 9   STREET TYPE             16920 non-null  object 
 10  CITY                    17998 non-null  object 
 11  STATE                   17593 non-null  object 
 12  ZIP CODE                17400 non-null  float64
 13  PRECINCT                17947 non-null  float64
 14  WARD PRECINCT           17998 non-null

In [64]:
df.fillna(df.mean, inplace=True)

C:\Users\quilt\AppData\Local\Temp\ipykernel_6848\3587490214.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<bound method DataFrame.mean of        PERMIT NUMBER  ACCOUNT NUMBER  SITE NUMBER              ISSUED DATE  \
0            1142666          458934            1  2019-11-25T00:00:00.000   
1            1142290          294900            4  2019-10-15T00:00:00.000   
2            1142132          459163            1  2019-09-20T00:00:00.000   
3            1142101          384478            1  2019-09-20T00:00:00.000   
4            1142100          456144            1  2019-09-19T00:00:00.000   
...              ...             ...          ...                      ...   
17993        1003612          204664            1  2001-08-01T00:00:00.000   
17994        1002993              51            1  2001-04-04T00:00:00.000   
17995        1002652          203574            1  2001-05-07T00:00:00.

In [65]:
df_processed = pd.get_dummies(df)

In [66]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17998 entries, 0 to 17997
Columns: 14457 entries, PERMIT NUMBER to Wards_6.0
dtypes: bool(14454), int64(3)
memory usage: 248.5 MB


In [67]:
train, test = train_test_split(df_processed, test_size=0.2, random_state=42)

In [68]:
y_train = train.index
y_test = test.index

In [71]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(train, y_train)

KNeighborsClassifier()

In [72]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [73]:
def calculate_distances(data, reference):
    return np.linalg.norm(data - reference, axis=1)

Testing

In [74]:
full_cafes = pd.read_csv("../data/sidewalk-cafe-permits.csv")
cafe_name = "SMALL CHEVAL"
reference_cafe = full_cafes[full_cafes["DOING BUSINESS AS NAME"] == cafe_name]

In [75]:
if reference_cafe.empty:
    print(f"El café '{cafe_name}' no se encuentra en la base de datos.")
else:
    index_cafe = reference_cafe.index.item()
    numeric_cafe = df_processed.iloc[index_cafe].values

    # Calcular distancias
    distances = calculate_distances(df_processed.values.astype(float), numeric_cafe.astype(float))
    recommended_indices = np.argsort(distances)[:5]

    # Mostrar las recomendaciones
    print("Recomendaciones:")
    for idx in recommended_indices[1:]:  # Excluir el primero, que es el mismo café
        recommended_cafe = full_cafes.iloc[idx]
        print(recommended_cafe["DOING BUSINESS AS NAME"])

Recomendaciones:
LA STRADA CAFE
RUNA JAPANESE
MELI CAFE ON DEARBORN
RANDOLPH TAVERN


In [56]:
df_processed.to_csv("../data/name_cafes.csv", index=False)